In [ ]:
import argparse
import os
import logging
import torch
import numpy as np

from tqdm import tqdm
from easydict import EasyDict

from models import build_model
from data import DataManger_Epoch, DataManger_Episode
from logger import setup_logging
from utils import read_config, rmdir, summary, array_interweave, COLOR
from evaluators import recognition_metrics

In [ ]:

def log_test(logger_func, attribute_name: list, weight, result_label, result_instance):
    r""" log test from result
    """
    logger_func('instance-based metrics:')
    logger_func('accuracy: %0.4f' % result_instance.accuracy)
    logger_func('precision: %0.4f' % result_instance.precision)
    logger_func('recall: %0.4f' % result_instance.recall)
    logger_func('f1_score: %0.4f' % result_instance.f1_score)

    logger_func('class-based metrics:')
    result = np.stack([result_label.accuracy, result_label.mean_accuracy, result_label.precision, result_label.recall, result_label.f1_score], axis=0)
    result = np.around(result*100, 2)
    result = result.transpose()
    row_format ="{:>20}" + "{:>10}"*6
    
    logger_func(row_format.format('attribute', 'weight', 'accuracy', 'mA', 'precision', 'recall', 'f1_score'))
    
    logger_func(row_format.format(*['-']*7))
    
    for i in range(len(attribute_name)):
        logger_func(row_format.format(attribute_name[i], np.around(weight[i]*100, 2), *result[i].tolist()))

    logger_func(row_format.format(*['-']*7))
    
    logger_func(row_format.format(
        'mean',
        '-',
        round(np.mean(result_label.accuracy)*100, 2),
        round(np.mean(result_label.mean_accuracy)*100, 2),
        round(np.mean(result_label.precision)*100, 2),
        round(np.mean(result_label.recall)*100, 2),
        round(np.mean(result_label.f1_score)*100, 2)))

In [ ]:

def compare_class_based(logger_func, attribute_name, weight, result_label1, result_label2, color=COLOR.BOLD):
    
    logger_func('class-based metrics:')
    result1 = np.stack([result_label1.accuracy, result_label1.mean_accuracy, result_label1.precision, result_label1.recall, result_label1.f1_score], axis=0)
    result1 = np.around(result1*100, 2)
    result1 = result1.transpose()

    result2 = np.stack([result_label2.accuracy, result_label2.mean_accuracy, result_label2.precision, result_label2.recall, result_label2.f1_score], axis=0)
    result2 = np.around(result2*100, 2)
    result2 = result2.transpose()

    row_format = "{:>20}{:>12}{:>15}{:>12}{:>20}{:>14}{:>18}"
    logger_func(row_format.format('attribute', 'weight', 'accuracy', 'mA', 'precision', 'recall', 'f1_score'))
    
    logger_func(row_format.format(*['-']*7))

    for i in range(len(attribute_name)):
        row_format = "{:>20}" + "{:>12}"
        for j in range(5):
            if result1[i][j] > result2[i][j]:
                row_format += color + "{:>10}" + COLOR.END +"|{:>5}"
            elif result1[i][j] < result2[i][j]:
                row_format += "{:>10}|" + color + "{:>5}" + COLOR.END
            else:
                row_format += color + "{:>10}" + COLOR.END + "|" + color + "{:>5}" + COLOR.END
        logger_func(row_format.format(attribute_name[i], np.around(weight[i]*100, 2), *array_interweave(result1[i], result2[i]).tolist()))

    row_format = "{:>20}" + "{:>12}" + "{:>10}|{:>5}"*5
    logger_func(row_format.format(*['-']*12))
    
    mean_result1 = np.around(np.mean(np.array([result_label1.accuracy, result_label1.mean_accuracy, result_label1.precision, result_label1.recall, result_label1.f1_score]), axis=1)*100, 2)
    
    mean_result2 = np.around(np.mean(np.array([result_label2.accuracy, result_label2.mean_accuracy, result_label2.precision, result_label2.recall, result_label2.f1_score]), axis=1)*100, 2)
    
    row_format = "{:>20}" + "{:>12}"
    for i in range(5):
        if mean_result1[i] > mean_result2[i]:
            row_format += color + "{:>10}" + COLOR.END +"|{:>5}"
        elif mean_result1[i] < mean_result2[i]:
            row_format += "{:>10}|" + color + "{:>5}" + COLOR.END
        else:
            row_format += color + "{:>10}" + COLOR.END + "|" + color + "{:>5}" + COLOR.END

    logger_func(row_format.format(
        'mean',
        '-',
        round(np.mean(result_label1.accuracy)*100, 2),
        round(np.mean(result_label2.accuracy)*100, 2),
        round(np.mean(result_label1.mean_accuracy)*100, 2),
        round(np.mean(result_label2.mean_accuracy)*100, 2),
        round(np.mean(result_label1.precision)*100, 2),
        round(np.mean(result_label2.precision)*100, 2),
        round(np.mean(result_label1.recall)*100, 2),
        round(np.mean(result_label2.recall)*100, 2),
        round(np.mean(result_label1.f1_score)*100, 2),
        round(np.mean(result_label2.f1_score)*100, 2)))

In [ ]:

def test(config, datamanager, logger_func):
    cfg_trainer = config['trainer_colab'] if config['colab'] == True else config['trainer']

    use_gpu = cfg_trainer['n_gpu'] > 0 and torch.cuda.is_available()
    device = torch.device('cuda:0' if use_gpu else 'cpu')
    map_location = "cuda:0" if use_gpu else torch.device('cpu')
    
    model, _ = build_model(config, num_classes=len(datamanager.datasource.get_attribute()))

    logger_func('Loading checkpoint: {} ...'.format(config['resume']))
    checkpoint = torch.load(config['resume'], map_location=map_location)

    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    model.to(device)
    
    preds = []
    labels = []

    with tqdm(total=len(datamanager.get_dataloader('test'))) as epoch_pbar:
        with torch.no_grad():
            for batch_idx, (data, _labels) in enumerate(datamanager.get_dataloader('test')):
                data, _labels = data.to(device), _labels.to(device)

                out = model(data)

                _preds = torch.sigmoid(out)
                preds.append(_preds)
                labels.append(_labels)
                epoch_pbar.update(1)
    preds = torch.cat(preds, dim=0)
    labels = torch.cat(labels, dim=0)
    preds = preds.cpu().numpy()
    labels = labels.cpu().numpy()

    result_label, result_instance = recognition_metrics(labels, preds)

    return result_label, result_instance

In [ ]:
config1 = "config/baseline_peta.yml"
config2 = "config/episode_peta.yml"

resume1 = "/content/drive/Shared drives/REID/HIEN/Models/OSNet_Person_Attribute_Refactor/checkpoints/0731_232453/model_best_accuracy.pth"
resume2 = "/content/drive/Shared drives/REID/HIEN/Models/person_attribute_recognition/checkpoints/0809_231322/model_best_accuracy.pth"

config1 = read_config(config1)
config1.update({'resume': resume1})
config1.update({'colab': True})

config2 = read_config(config2)
config2.update({'resume': resume2})
config2.update({'colab': True})

datamanager1 = DataManger_Epoch(config1['data'])
datamanager2 = DataManger_Episode(config2['data'])

weight = datamanager1.datasource.get_weight('train')

# model1
result_label1, result_instance1 = test(config1, datamanager1, print)

# model 2
result_label2, result_instance2 = test(config2, datamanager2, print)

In [ ]:
compare_class_based(print, datamanager1.datasource.get_attribute(), weight, result_label1, result_label2, COLOR.BLUE)

In [ ]:
weight = datamanager1.datasource.get_weight('train')
print(weight)

In [ ]:
from torch.distributions.multinomial import Multinomial
m = Multinomial(32, torch.exp(1-torch.tensor(weight)))  

In [ ]:
attribute_name = datamanager1.datasource.get_attribute()

In [ ]:
import random
selected_attribute = random.sample(attribute_name, 8)
print(selected_attribute)

In [ ]:
m.sample()

In [ ]:
Multinomial(probs=torch.exp(1-torch.tensor(weight))).sample(torch.Size([8]))

In [ ]:
torch.multinomial(torch.exp(1-torch.tensor(weight)), 8, replacement=True)

In [ ]:
import collections
from collections import defaultdict
result = defaultdict(int)
for _ in range(100000):
    temp = torch.multinomial(torch.exp(1-torch.tensor(weight)), 8, replacement=True)
    for key, value in collections.Counter(temp.numpy()).items():
        result[key] += value
print(list(collections.OrderedDict(sorted(result.items())).values()))